In [ ]:
import paho.mqtt.client as mqtt
import time
import random
import json

client = mqtt.Client()
client.connect("localhost", 1883, 60)

machine_id = "machine_01"
steps = {
    "step_1": {"mean": 50, "variance": 5},
    "step_2": {"mean": 30, "variance": 3},
}

while True:
    for step, stats in steps.items():
        measurement = random.gauss(stats["mean"], stats["variance"])
        payload = json.dumps({
            "step_id": step,
            "value": measurement,
            "mean": stats["mean"],
            "variance": stats["variance"],
            "timestamp": time.strftime('%Y-%m-%d %H:%M:%S')
        })
        topic = f"factory/{machine_id}/{step}"
        client.publish(topic, payload)
    time.sleep(2)
